[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openscilabs/isda/blob/main/benchmark.ipynb)

# MISDA - Maximal Independent Structural Dimensionality Analysis

This notebook serves as a comprehensive benchmark suite for the Maximal Independent Structural Dimensionality Analysis (MISDA) framework.
It systematically evaluates the algorithm's efficacy across a spectrum of synthetic benchmarks, ranging from canonical correlation patterns—including linear redundancies and latent manifolds—to complex Multi-Objective Problems (MOPs). The analysis verifies MISDA's capability to correctly identify intrinsic dimensionality and preserve the topological fidelity of the Pareto frontier.

---
**Copyright (C) 2025 Monaco F. J.** <monaco@usp.br>

This program is free software: you can redistribute it and/or modify it under the terms of the **GNU General Public License** as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

SPDX-License-Identifier: GPL-3.0-or-later
---

In [ ]:
# Install MISDA 
!pip install --upgrade git+https://github.com/openscilabs/isda.git

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import misda

# Reload for development iteration
import importlib
importlib.reload(misda)

print("MISDA imported successfully.")

## 1. Data Generators & Utilities

### General Helpers

In [ ]:
def _truth(name, intrinsic_dim_expected, blocks_expected, notes=""):
    return {
        "name": name,
        "intrinsic_dim_expected": int(intrinsic_dim_expected),
        "blocks_expected": blocks_expected,
        "notes": notes,
    }

def _mk_block_names(start, size):
    # start is 1-based
    return [f"f{i}" for i in range(start, start + size)]

def _repeat_with_small_noise(base, rng, noise):
    # base: (N,) -> returns perturbed (N,)
    return base + noise * rng.normal(size=base.shape[0])

def _mop_df(Y):
    return pd.DataFrame(Y, columns=[f"f{i+1}" for i in range(Y.shape[1])])

### Evaluation Utility

In [ ]:
# Validation Utility: misda.compile_benchmark_summary is used instead.


### MOP Generators

In [ ]:
def mopA_linear_redundancy_3_groups(N=1000, seed=123, noise=0.05):
    rng = np.random.default_rng(seed)
    # Latents
    X1 = rng.uniform(0.0, 1.0, size=N)
    X2 = rng.uniform(0.0, 1.0, size=N)
    X3 = rng.uniform(0.0, 1.0, size=N)
    
    # Group 1 (around X1) - size 5
    G1 = [X1]
    for _ in range(4):
        G1.append(_repeat_with_small_noise(X1, rng, noise))
        
    # Group 2 (around X2) - size 5
    G2 = [X2]
    for _ in range(4):
        G2.append(_repeat_with_small_noise(X2, rng, noise))
        
    # Group 3 (around X3) - size 10
    G3 = [X3]
    for _ in range(9):
        G3.append(_repeat_with_small_noise(X3, rng, noise))
        
    # Concatenate
    # Total M = 5 + 5 + 10 = 20
    feats = G1 + G2 + G3
    Y = np.vstack(feats).T
    
    truth = _truth(
        name="MOP-A — Linear Redundancy (3 groups, M=20)",
        intrinsic_dim_expected=3,
        blocks_expected=[_mk_block_names(1, 5), _mk_block_names(6, 5), _mk_block_names(11, 10)],
        notes="Simple linear correlation + small noise. Expecting 3 MIS elements."
    )
    return _mop_df(Y), truth

def mopB_tradeoff_with_redundancies(N=1000, seed=123, noise=0.03):
    rng = np.random.default_rng(seed)
    # Base tradeoff P vs Q (approx circle)
    theta = rng.uniform(0, np.pi/2, size=N)
    P = np.cos(theta)
    Q = np.sin(theta)
    
    # 7 "cost" objectives (driven by P)
    P_rep = np.clip(_repeat_with_small_noise(P, rng, noise), 0.0, 1.0)
    
    cost_feats = [
        P,
        P_rep,
        P**2,
        np.sqrt(np.maximum(P, 0.0)),
        np.log1p(9.0 * P),
        (P + 0.1) ** 1.5,
        1.0 / (1.0 + np.exp(-10*(P-0.5))) # sigmoid
    ]
    
    # 7 "consumption" objectives (driven by E = 1-P approx Q)
    # Let's use E = 1 - P as a base, strongly correlated to Q but linear to P
    E = 1.0 - P
    E_rep = np.clip(_repeat_with_small_noise(E, rng, noise), 0.0, 1.0)
    
    cons_feats = [
        E,
        E_rep,
        np.exp(E) - 1.0,
        np.tanh(E),
        E**2,
        (E + 0.05),
        (E + 0.2) ** 1.3,
    ]
    
    # 6 "performance" objectives (minimization via 1-P), with protected domain
    Q_rep = np.clip(_repeat_with_small_noise(Q, rng, noise), 0.0, 1.0)
    
    perf_feats = [
        Q,
        Q_rep,
        Q**2,
        np.sqrt(np.maximum(Q, 0.0)),
        np.log1p(9.0 * Q),            # now Q ∈ [0,1] -> always valid
        (Q + 0.1) ** 1.2,             # now Q+0.1 ∈ [0.1,1.1] -> always valid
    ]
    
    feats = cost_feats + cons_feats + perf_feats
    Y = np.vstack(feats).T
    
    truth = _truth(
        name="MOP-B — Trade-off + redundancies (~2D, M=20)",
        intrinsic_dim_expected=2,
        blocks_expected=[_mk_block_names(1, 7), _mk_block_names(8, 7), _mk_block_names(15, 6)],
        notes="Three families (cost/consumption/performance) with internal redundancies; effective tends to ~2."
    )
    return _mop_df(Y), truth

def mopC_latent_blocks_4x5(N=1000, seed=123, noise=0.02):
    rng = np.random.default_rng(seed)
    u, v, w, z = rng.uniform(0.0, 1.0, size=(4, N))
    eps = rng.normal(size=N)
    
    b1 = [u, 2*u, u**2, np.sqrt(np.maximum(u,0.0)), np.log1p(9*u)]
    b2 = [v, v+0.5, np.log1p(9*v), v**2, np.sqrt(np.maximum(v,0.0))]
    b3 = [w, w+noise*eps, np.sqrt(np.maximum(w,0.0)), np.log1p(9*w), (w+0.1)**2]
    b4 = [z, (1.0+z)**2, np.exp(z)-1.0, np.log1p(9*z), np.sqrt(np.maximum(z,0.0))]
    
    feats = b1 + b2 + b3 + b4
    Y = np.vstack(feats).T
    
    truth = _truth(
        name="MOP-C — Latent blocks (4×5, M=20)",
        intrinsic_dim_expected=4,
        blocks_expected=[_mk_block_names(1,5), _mk_block_names(6,5), _mk_block_names(11,5), _mk_block_names(16,5)],
        notes="Four independent factors; each block (5 objectives) is internally redundant."
    )
    return _mop_df(Y), truth

def mopD_pure_conflict_groups(N=1000, seed=123, noise=0.0):
    rng = np.random.default_rng(seed)
    x = rng.uniform(0.0, 1.0, size=N)
    
    g1 = [
        x,
        2*x + 0.1,
        np.log1p(9*x),
        x**2,
        np.sqrt(np.maximum(x,0.0)),
        x**3,
        np.tanh(2*x),
        np.log1p(3*x),
        (x+0.2)**2,
        (1.0 + x)**1.5,
    ]
    y = 1.0 - x
    g2 = [
        y,
        2*y + 0.1,
        np.log1p(9*y),
        y**2,
        np.sqrt(np.maximum(y,0.0)),
        y**3,
        np.tanh(2*y),
        np.log1p(3*y),
        (y+0.2)**2,
        (1.0 + y)**1.5,
    ]
    
    feats = [_repeat_with_small_noise(f, rng, noise) for f in (g1 + g2)]
    Y = np.vstack(feats).T
    
    truth = _truth(
        name="MOP-D — Structural conflict (anti-corr) 2-groups (M=20)",
        intrinsic_dim_expected=2,
        blocks_expected=[_mk_block_names(1,10), _mk_block_names(11,10)],
        notes="Two internally redundant groups (+x and 1-x), but antagonistic to each other: conflict must be preserved."
    )
    return _mop_df(Y), truth

def mopE_partial_redundancy_noisy(N=1000, seed=123, noise=0.05):
    rng = np.random.default_rng(seed)
    a = rng.uniform(0.0, 1.0, size=N)
    b = rng.uniform(0.0, 1.0, size=N)
    eps = rng.normal(size=N)
    
    # subfamily A: redundant around 'a' (10)
    A = [
        a,
        a + noise*eps,
        a - noise*eps,
        a**2,
        np.sqrt(np.maximum(a,0.0)),
        np.log1p(a),
        np.exp(a)-1,
        (a+0.1)**2,
        1.0/(1.0+np.exp(-a)),
        a + 0.1*noise*rng.normal(size=N)
    ]
    
    # subfamily B: redundant around 'b' (4)
    B = [
        b,
        b**2,
        np.log1p(b),
        np.sqrt(np.maximum(b,0.0))
    ]
    
    # subfamily C: redundant around 's = a+b' (6)
    # this creates a bridge or a larger structure
    s = a + b
    eps2 = rng.normal(size=N)
    C = [
        s,
        s + noise*eps2,
        s**2,
        np.log1p(s),
        np.sqrt(np.maximum(s,0.0)),
        (s+0.1)**1.5
    ]
    
    feats = A + B + C
    Y = np.vstack(feats).T
    
    truth = _truth(
        name="MOP-E — Partial redundancy + noise (M=20)",
        intrinsic_dim_expected=2,
        blocks_expected=[_mk_block_names(1,10), _mk_block_names(11,4), _mk_block_names(15,6)],
        notes="Trio/quartet of 'a' extended to 10 redundants; 'b' (4); and 6 compounds around s=a+b."
    )
    return _mop_df(Y), truth

def mopF_regime_switching(N=1000, seed=123, sharpness=20.0, noise=0.0):
    rng = np.random.default_rng(seed)
    a = rng.uniform(0.0, 1.0, size=N)
    b = rng.uniform(0.0, 1.0, size=N)
    
    s = 1.0 / (1.0 + np.exp(-sharpness * (a - 0.5)))
    L = (1.0 - s) * a + s * b
    
    eps = rng.normal(size=N)
    
    L_feats = [
        L,
        L**2,
        np.log1p(9*L),
        np.sqrt(np.maximum(L,0.0)),
        (L+0.1)**1.5,
        np.tanh(2*L),
        np.exp(0.5*L)-1.0,
        (L+0.2)**2,
        np.log1p(3*L),
        _repeat_with_small_noise(L, rng, 0.02) if noise == 0.0 else _repeat_with_small_noise(L, rng, noise),
    ]
    
    b_feats = [
        b,
        np.sqrt(np.maximum(b,0.0)),
        np.log1p(9*b),
        b**2,
        (b+0.1)**1.5,
        np.tanh(2*b),
        np.exp(0.5*b)-1.0,
        (b+0.2)**2,
        np.log1p(3*b),
        _repeat_with_small_noise(b, rng, 0.02) if noise == 0.0 else _repeat_with_small_noise(b, rng, noise),
    ]
    
    feats = L_feats + b_feats
    Y = np.vstack(feats).T
    
    truth = _truth(
        name="MOP-F — Regimes (mixture, M=20)",
        intrinsic_dim_expected=2,
        blocks_expected=[_mk_block_names(1,10), _mk_block_names(11,10)],
        notes="Variable correlation structure switched by sigmoid. Checks if global method breaks or finds the two drivers."
    )
    return _mop_df(Y), truth

## 2. Execution & Analysis

In [ ]:
datasets = {}

# Generate data
datasets["MOP-A"] = mopA_linear_redundancy_3_groups()
datasets["MOP-B"] = mopB_tradeoff_with_redundancies()
datasets["MOP-C"] = mopC_latent_blocks_4x5()
datasets["MOP-D"] = mopD_pure_conflict_groups()
datasets["MOP-E"] = mopE_partial_redundancy_noisy()
datasets["MOP-F"] = mopF_regime_switching()

results = {}

print("Starting Benchmark Execution...")
for key, (df, truth) in datasets.items():
    print(f"\nProcessing {key}...")
    # Default caution=0.5
    res = misda.analyze(df, caution=0.5, run_ses=True, name=truth.get("name", key))
    results[key] = {
        "result_obj": res,
        "truth": truth
    }
    
print("\n--- Benchmark Complete ---")

## 3. Performance Summary

In [ ]:
print("\n=== BENCHMARK SUMMARY ===")
df_summary = misda.compile_benchmark_summary(results)
print(df_summary.to_string(index=False))
